In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


> Deepseek api测试

上下文拼接：

对于推理模型，比如deepseek r1来说。在每一轮对话过程中，模型会输出思维链内容和最终回答，在下一轮d对话中，之前轮输的思维链内容不会被拼接到上下文中

<img src="../assets/cot.png" alt="examples" style="zoom:50%;" /> 

In [2]:
# 创建clent
from openai import OpenAI
from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key,base_url="https://api.deepseek.com")

In [8]:
# 推理模型，输出思维链内容
# 非流式

# Round 1
messages = [{"role": "user", "content": "你好呀"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

reasoning_content = response.choices[0].message.reasoning_content
content = response_content = response.choices[0].message.content

# Round 2
messages.append({'role': 'assistant', 'content': content})
messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

reasoning_content = response.choices[0].message.reasoning_content
reasoning_content

'Okay, let\'s see. The user is asking how many times the letter \'R\' appears in the word \'strawberry\'. Hmm, first I need to make sure I spell the word correctly. Strawberry. Let me break it down. S-T-R-A-W-B-E-R-R-Y. Wait, is that right? Let me check again.\n\nSo, starting with S, then T, R, A, W, B, E, R, R, Y. Let me count the Rs. The third letter is R, and then after B and E, there are two Rs before the Y. So that\'s one R in the beginning and two Rs towards the end. Wait, no. Wait, the spelling is S-T-R-A-W-B-E-R-R-Y. So after the W comes B, E, then R, R, Y. So the Rs are at the third position and then the seventh and eighth positions? Let me count again.\n\nLetters: S (1), T (2), R (3), A (4), W (5), B (6), E (7), R (8), R (9), Y (10). So positions 3, 8, and 9? Wait, that can\'t be right. Because after B (6) comes E (7), then R (8), R (9), Y (10). So the Rs are at positions 3, 8, and 9? That would be three Rs. But wait, the word is strawberry. Let me confirm the correct spellin

In [9]:
# 流式
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

reasoning_content = ""
for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content
        print(chunk.choices[0].delta.reasoning_content, end="", flush=True)

Okay, let's tackle this question: "How many Rs are there in the word 'strawberry'?" Hmm, first I need to make sure I spell the word correctly. Strawberry. Let me break it down letter by letter.

S-T-R-A-W-B-E-R-R-Y. Wait, is that right? Let me write it out: S, T, R, A, W, B, E, R, R, Y. Let me count the letters again to confirm. S (1), T (2), R (3), A (4), W (5), B (6), E (7), R (8), R (9), Y (10). So the word has 10 letters. But the user is asking specifically about the number of Rs.

Looking at the letters: R is the third letter. Then after B and E, there's R again. Wait, is that two Rs at the end before Y? Let me check again. Breaking it down:

1. S
2. T
3. R
4. A
5. W
6. B
7. E
8. R
9. R
10. Y

So positions 3, 8, and 9 are R, R, and R? Wait, that can't be right. Wait, maybe I made a mistake in the breakdown. Let me think again. The correct spelling of strawberry is S-T-R-A-W-B-E-R-R-Y. So after B comes E, then two Rs, and then Y. So the letters after B are E, R, R, Y. So that's two

In [4]:
# 多轮对话
# Round 1
messages = [{"role": "user", "content": "What's the highest mountain in the world?"}]
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)

msg = response.choices[0].message

messages.append({"role": msg.role, "content": msg.content})
messages

[{'role': 'user', 'content': "What's the highest mountain in the world?"},
 {'role': 'assistant',
  'content': 'The highest mountain in the world is **Mount Everest**, located in the **Mahalangur Himal** sub-range of the **Himalayas** on the border between **Nepal** and **Tibet (China)**.  \n\n- **Elevation:** **8,848.86 meters (29,031.7 feet)** above sea level (as per the latest 2020 survey by Nepal and China).  \n- **First Ascent:** Sir **Edmund Hillary** (New Zealand) and **Tenzing Norgay** (Nepal/India) reached the summit on **May 29, 1953**.  \n\nMount Everest is also known as:  \n- **Sagarmatha** (in Nepal)  \n- **Chomolungma** (in Tibet, meaning "Goddess Mother of the World")  \n\nWould you like details on climbing routes, records, or other extreme mountains?'}]

In [ ]:
# function calling tools

messages = [
    {"role": "user", "content": "What's the weather like in Boston?"},
]

def get_weather(location: str):
    """获取当前天气"""
    return {
        "location": location,
        "temperature": "22°C",
        "condition": "Sunny"
    }

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

def get_response(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools,
    )
    return response

response = get_response(messages)

# 需要把返回的内容添加到消息列表中
messages.append({
    "role": "assistant",
    "content": None,  # Content is None when there are tool calls
    "tool_calls": response.choices[0].message.tool_calls
})

tool = response.choices[0].message.tool_calls[0]

# 调用工具
if response.choices[0].message.tool_calls:
    function_name = response.choices[0].message.tool_calls[0].function.name
    print(function_name)
    if function_name == "get_weather":
        location = eval(response.choices[0].message.tool_calls[0].function.arguments)["location"]
        weather_info = get_weather(location)
        print(weather_info)
messages.append({"role": "tool", "tool_call_id": tool.id, "content": str(weather_info)})
# 发送更新后的消息列表
response = get_response(messages)
print(response.choices[0].message.content)

get_weather
{'location': 'Boston', 'temperature': '22°C', 'condition': 'Sunny'}
The weather in Boston is currently sunny with a temperature of 22°C.


In [ ]:
# push代码使出现了Failed to connect to github.com port 443 after 75002 ms
# 解决方法，执行下面的两个命令
# 删除https代理设置
git config --global --unset https.proxy
# 删除http代理设置
git config --global --unset http.proxy